<a href="https://colab.research.google.com/github/lefkothea1/NLP_2/blob/main/Copy_of_NLP_ass2_ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
#to select GPU: Edit → Notebook Settings

#loading data from github repo- WILL USE GOOGLE DRIVE
# import pandas as pd
# url_diagn_test = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv'
# url_test = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-test.csv'
# url_train ='https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-train.csv'
# diagnostic_test = pd.read_csv(url_test)
# train = pd.read_csv(url_train)
# test = pd.read_csv(url_test)

#setting up google drive and loading all data
from google.colab import drive

drive.mount('/content/drive')
train = pd.read_csv("/content/drive/MyDrive/NLP2/data/Assignment 2_IntrotoNLP2022_data/olid-train.csv")
test = pd.read_csv("/content/drive/MyDrive/NLP2/data/Assignment 2_IntrotoNLP2022_data/olid-test.csv")
diagn_test = pd.read_csv("/content/drive/MyDrive/NLP2/data/Assignment 2_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv")

print(train.shape)
print(train.head())
#train set is already in the format needed by the model !

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(13240, 3)
      id                                               text  labels
0  86426  @USER She should ask a few native Americans wh...       1
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       1
2  16820  Amazon is investigating Chinese employees who ...       0
3  62688  @USER Someone should'veTaken" this piece of sh...       1
4  43605  @USER @USER Obama wanted liberals &amp; illega...       0


In [28]:
#setting up- from https://simpletransformers.ai/docs/binary-classification/--------dunno where to find this log
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [29]:
#should i install it every time?-no
#!pip install simpletransformers 

In [30]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [33]:
# Optional model configuration-using best teams args -from paper- SHOULD I USE GRID THINGY TO CHECK?DIDNT FOR NOW
model_args = ClassificationArgs()
model_args.num_train_epochs=2
# model_args.max_sentence_length=64-not sure about this one
model_args.output_dir = '/content/drive/MyDrive/NLP2/trained_models'

In [35]:
#loading neccessary pretrained model -following steps only needed during first run. then just load the saved model
#bert_mod = ClassificationModel('bert', 'bert-base-cased', args = model_args)
#training/ finetuning on OUR training data-takes time! no need to run it again
#bert_mod.train_model(train)

#loading finetuned MODEL FROM DRIVE
bert_mod = ClassificationModel("bert", "/content/drive/MyDrive/NLP2/trained_models")

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/13240 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1655 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1655 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to /content/drive/MyDrive/NLP2/trained_models.


(3310, 0.4878392012097684)

In [38]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

# Evaluating the model
result, model_outputs, wrong_predictions = bert_mod.eval_model(test, clf_report = classification_report)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/860 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.5809657009127254, 'tp': 140, 'tn': 583, 'fp': 37, 'fn': 100, 'auroc': 0.8708434139784946, 'auprc': 0.7546967612136047, 'clf_report': '              precision    recall  f1-score   support\n\n         0.0       0.85      0.94      0.89       620\n         1.0       0.79      0.58      0.67       240\n\n    accuracy                           0.84       860\n   macro avg       0.82      0.76      0.78       860\nweighted avg       0.84      0.84      0.83       860\n', 'precision': 0.7909604519774012, 'eval_loss': 0.4053518639670478}


In [46]:
print(result['clf_report'])
#model_outputs
#wrong_predictions
print(result) #target class is 1 (offensive)

              precision    recall  f1-score   support

         0.0       0.85      0.94      0.89       620
         1.0       0.79      0.58      0.67       240

    accuracy                           0.84       860
   macro avg       0.82      0.76      0.78       860
weighted avg       0.84      0.84      0.83       860

{'mcc': 0.5809657009127254, 'tp': 140, 'tn': 583, 'fp': 37, 'fn': 100, 'auroc': 0.8708434139784946, 'auprc': 0.7546967612136047, 'clf_report': '              precision    recall  f1-score   support\n\n         0.0       0.85      0.94      0.89       620\n         1.0       0.79      0.58      0.67       240\n\n    accuracy                           0.84       860\n   macro avg       0.82      0.76      0.78       860\nweighted avg       0.84      0.84      0.83       860\n', 'precision': 0.7909604519774012, 'eval_loss': 0.4053518639670478}


In [49]:
print(140/(140+37))
print(result['precision'])

0.7909604519774012
0.7909604519774012
